##Clean the dataframe with information from Wikipedia.##
The task is to clean the column containing box office information.

In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import re
import string
import json
from tqdm import tqdm

In [4]:
df = pd.read_pickle('wiki_df_raw.csv')

In [5]:
df[df['rt_link']=="NA"].shape

(515, 6)

In [6]:
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/


In [7]:
#test
df['budget'][1262]
df['budget'][527][0]
df['budget'][527][0]

[u'\xa387,000']

Check that all movies each has only one information in budget. It should be eigher the budget information or "NA" for lack of information on Wikipedia.

In [7]:
for item in df['budget']:
    if len(item)!=1:
        print item

Cleaning process. 
There are several challenges because the format is very different. 

All are coded in unicode, but they have different format, for example,"\$40–50 million", "\$1.1 million","₤4 million" and so on. The unicode for "–" symble in "\$40–50 million" cannot be successfully extracted. Because there are only around 10 of these cases, we treate them as "NA".

The pound ₤ sign is also coded in unicode, but we found a way to take them out.

There are other different currencies, so we added a list named ```currency```.

If there are other errors in the process, we coded them as some form of "ERR".
Because there are relatively low numbers of these errors and they are difficult to fix, we also treat them as "NA".

In [171]:
budget_sorted=[]
currency=[]
figure_error_1_index=[]
figure_error_2_index=[]
currency_error_index=[]
figure_error_index=[]
error=[]
x=[]
mm=0
for item in df['budget']:
    mm=mm+1
    #print mm-1
    for i in item:
        if mm in [278+1,344+1,403+1,427+1,487+1,527+1,892+1,896+1,990+1,1059+1]: #all those of the form $40–50 million
            currency.append("FERR1")
            budget_sorted.append('FERR1')
            #print i
            figure_error_1_index.append(mm-1)
            ###################################
        else:
            #print i
            if i=="NA": #takes care of the NA budgets
                budget_sorted.append("NA")
                currency.append("NA")
                
            else:
                if len(i.split("$"))==2: #all those of the form $1.1 million
                    j=[]
                    i=i.split("$")
                    j.append(i[0]) # currency marker
                    j=j+(i[1].split(" "))
                    #print j       # j has the form [u'', u'1,453,000'] or [u'', u'12', u'million']
                elif len(i.split("$"))==1: #all those of the form ₤4 million
                    ii=i.encode('ascii','ignore') #get the money figure in the "₤4 million"
                    j=["P"]
                    j=j+ii.split(" ")
                    #print j     # j has the form [u'P', u'1,453,000'] or [u'P', u'12', u'million']
                
                #takes care of the currency part
                if j[0]=='':
                    currency.append("USD")
                elif j[0]=="A":
                    currency.append("AUD")
                elif j[0]=="US" or j[0]=="(":
                    currency.append("USD")   
                elif j[0]=="P":
                    currency.append("GBP")  
                else:
                    currency_error_index.append(mm-1)
                    #print j,mm-1
                    currency.append("CERR")
                
                
                if len(j)==2:
                    budget_sorted.append(j[1].replace(",",""))
                #else:
                    #figure_error_2_index.append(mm-1)
                    #budget_sorted.append("FERR2") 
                elif 'million' in j:
                    unit=1000000
                    budget_sorted.append(int(float(j[1])*unit))
                else:
                    #print i
                    figure_error_index.append(mm-1)
                    budget_sorted.append("FERR")

In [172]:
len(figure_error_1_index),len(figure_error_2_index),len(figure_error_index),len(currency_error_index),df[df.budget_sorted=="NA"].shape[0]

(10, 0, 23, 11, 390)

So in totle, we have 390+10+23+11=434 "NA"'s out of 1276 entries.

In [173]:
len(budget_sorted),len(currency)

(1275, 1275)

In [178]:
for m,b in enumerate(budget_sorted):
    if m not in [158,328,349,401,440,444,468,607,608,632,765,963,968,984,1052,1265]:
        if b not in ["NA","FERR","CERR","FERR1"]:
            '''#print m
            #print float(b)'''

In [175]:
for i in [158,328,349,401,440,444,468,607,608,632,765,963,968,984,1052,1265]:
    #print budget_sorted[i]
    budget_sorted[i]=budget_sorted[i].encode('ascii','ignore')
    if budget_sorted[i] in ["N/A","NA","Unidentified"]:
        budget_sorted[i]="NA"
    else:
        j=budget_sorted[i].split("m")
        budget_sorted[i]=int(float(j[0])*1000000)

In [188]:
for m,b in enumerate(budget_sorted):
    if b in ["NA","FERR","CERR","FERR1"]:
        budget_sorted[m]="NA"
    else:
        budget_sorted[m]=float(budget_sorted[m])

for m,b in enumerate(currency):
    if b in ["NA","FERR","CERR","FERR1"]:
        currency[m]="NA"
    else:
        currency[m]=currency[m]
    


In [189]:
#budget_sorted

We merge them into dataset.

In [190]:
df['budget_sorted']=budget_sorted
df['currency']=currency
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link,budget_sorted,currency
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...,6000000,AUD
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA,1750000,USD
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...,NA,NA
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA,NA,NA
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/,7500000,USD


Extract the imdb if for merging purpose later.

In [191]:
imdb_id=[]
for l in df.imdb_link:
    if l!="NA":
        imdb_id.append(l.split("/")[-2])
    else:
        imdb_id.append("NA")

In [192]:
df['imdb_id']=imdb_id
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link,budget_sorted,currency,imdb_id
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...,6000000,AUD,tt0086617
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA,1750000,USD,tt0038053
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...,NA,NA,tt041113
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA,NA,NA,tt0021310
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/,7500000,USD,tt1174732


In [193]:
df.to_pickle('wiki_df_cleaned.csv')